In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [1]:
%cd /content/drive/MyDrive/HYF/Graduation_Fin/FinRL
## install finrl library
# !pip install git+https://github.com/AI4Finance-LLC/FinRL-Library.git
!pip install -r requirements.txt

/content/drive/MyDrive/HYF/Graduation_Fin/FinRL
  Using cached pyfolio-0.9.2-py3-none-any.whl
  Using cached empyrical-0.5.5-py3-none-any.whl
  Attempting uninstall: empyrical
    Found existing installation: empyrical 0.3.4
    Uninstalling empyrical-0.3.4:
      Successfully uninstalled empyrical-0.3.4
  Attempting uninstall: pyfolio
    Found existing installation: pyfolio 0.8.0
    Uninstalling pyfolio-0.8.0:
      Successfully uninstalled pyfolio-0.8.0


# 思路
1. 需要随机初始点，但是test_env中是利用上一次的初始状态
2. 直接更改reward规则来约束exposure
3. 更改卖空机制以及金钱下线
4. 可能需要downround（在计算购买股票的钱的时候，防止upround直接爆balance）


# 结构
[PPO.train()](https://github.com/DLR-RM/stable-baselines3/blob/75b6f3b3b0f207456d9dcac2c6e86e8e2a22115f/stable_baselines3/ppo/ppo.py#L181)

[OnPolicyAlgorithm.rollout_buffer](https://github.com/DLR-RM/stable-baselines3/blob/75b6f3b3b0f207456d9dcac2c6e86e8e2a22115f/stable_baselines3/common/on_policy_algorithm.py#L112)

[RolloutBuffer]()

无效，太深了，不能改



#git

In [56]:
!git status

On branch master
Your branch is up to date with 'origin/master'.

Changes not staged for commit:
  (use "git add <file>..." to update what will be committed)
  (use "git checkout -- <file>..." to discard changes in working directory)

	modified:   FinRL_StockTrading_NeurIPS_2018.ipynb
	modified:   finrl/finrl_meta/env_stock_trading/env_test.py
	modified:   requirements.txt

no changes added to commit (use "git add" and/or "git commit -a")


In [ ]:
# !git add finrl/finrl_meta/env_stock_trading/env_test.py

!git add -u

In [ ]:
!git config --global user.email "939778128@qq.com"
!git config --global user.name "EvanHong99"
!git commit -m "synchronize"

[master 586d3ad] synchronize
 2 files changed, 25 insertions(+), 39 deletions(-)
 rewrite FinRL_StockTrading_NeurIPS_2018.ipynb (93%)


In [ ]:
!git pull

Already up to date.


In [ ]:
!git push

Counting objects: 7, done.
Delta compression using up to 2 threads.
Compressing objects: 100% (7/7), done.
Writing objects: 100% (7/7), 3.91 KiB | 267.00 KiB/s, done.
Total 7 (delta 6), reused 0 (delta 0)
remote: Resolving deltas: 100% (6/6), completed with 6 local objects.
To https://github.com/EvanHong99/FinRL.git
   a55d7f1..586d3ad  master -> master


<a href="https://colab.research.google.com/github/AI4Finance-Foundation/FinRL/blob/master/FinRL_StockTrading_NeurIPS_2018.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Deep Reinforcement Learning for Stock Trading from Scratch: Multiple Stock Trading

* **Pytorch Version** 



# Content

* [1. Problem Definition](#0)
* [2. Getting Started - Load Python packages](#1)
    * [2.1. Install Packages](#1.1)    
    * [2.2. Check Additional Packages](#1.2)
    * [2.3. Import Packages](#1.3)
    * [2.4. Create Folders](#1.4)
* [3. Download Data](#2)
* [4. Preprocess Data](#3)        
    * [4.1. Technical Indicators](#3.1)
    * [4.2. Perform Feature Engineering](#3.2)
* [5.Build Environment](#4)  
    * [5.1. Training & Trade Data Split](#4.1)
    * [5.2. User-defined Environment](#4.2)   
    * [5.3. Initialize Environment](#4.3)    
* [6.Implement DRL Algorithms](#5)  
* [7.Backtesting Performance](#6)  
    * [7.1. BackTestStats](#6.1)
    * [7.2. BackTestPlot](#6.2)   
    * [7.3. Baseline Stats](#6.3)   
    * [7.3. Compare to Stock Market Index](#6.4)   
* [RLlib Section](#7)            

<a id='0'></a>
# Part 1. Problem Definition

This problem is to design an automated trading solution for single stock trading. We model the stock trading process as a Markov Decision Process (MDP). We then formulate our trading goal as a maximization problem.

The algorithm is trained using Deep Reinforcement Learning (DRL) algorithms and the components of the reinforcement learning environment are:


* Action: The action space describes the allowed actions that the agent interacts with the
environment. Normally, a ∈ A includes three actions: a ∈ {−1, 0, 1}, where −1, 0, 1 represent
selling, holding, and buying one stock. Also, an action can be carried upon multiple shares. We use
an action space {−k, ..., −1, 0, 1, ..., k}, where k denotes the number of shares. For example, "Buy
10 shares of AAPL" or "Sell 10 shares of AAPL" are 10 or −10, respectively

* Reward function: r(s, a, s′) is the incentive mechanism for an agent to learn a better action. The change of the portfolio value when action a is taken at state s and arriving at new state s',  i.e., r(s, a, s′) = v′ − v, where v′ and v represent the portfolio
values at state s′ and s, respectively

* State: The state space describes the observations that the agent receives from the environment. Just as a human trader needs to analyze various information before executing a trade, so
our trading agent observes many different features to better learn in an interactive environment.

* Environment: Dow 30 consituents


The data of the single stock that we will be using for this case study is obtained from Yahoo Finance API. The data contains Open-High-Low-Close price and volume.


<a id='1'></a>
# Part 2. Getting Started- Load Python Packages

<a id='1.1'></a>
## 2.1. Install all the packages through FinRL library


In [ ]:
%cd /content/drive/MyDrive/HYF/Graduation_Fin/FinRL
root='/content/drive/MyDrive/HYF/Graduation_Fin/FinRL'
from finrl import config
from finrl import config_tickers
import os
if not os.path.exists(root+"/" + config.DATA_SAVE_DIR):
    os.makedirs(root+"/" + config.DATA_SAVE_DIR)
if not os.path.exists(root+"/" + config.TRAINED_MODEL_DIR):
    os.makedirs(root+"/" + config.TRAINED_MODEL_DIR)
if not os.path.exists(root+"/" + config.TENSORBOARD_LOG_DIR):
    os.makedirs(root+"/" + config.TENSORBOARD_LOG_DIR)
if not os.path.exists(root+"/" + config.RESULTS_DIR):
    os.makedirs(root+"/" + config.RESULTS_DIR)

/content/drive/MyDrive/HYF/Graduation_Fin/FinRL



<a id='1.2'></a>
## 2.2. Check if the additional packages needed are present, if not install them. 
* Yahoo Finance API
* pandas
* numpy
* matplotlib
* stockstats
* OpenAI gym
* stable-baselines
* tensorflow
* pyfolio

<a id='1.3'></a>
## 2.3. Import Packages

In [ ]:
import pandas as pd
import numpy as np
import matplotlib
import matplotlib.pyplot as plt
# matplotlib.use('Agg')
import datetime

%matplotlib inline
from finrl.finrl_meta.preprocessor.yahoodownloader import YahooDownloader
from finrl.finrl_meta.preprocessor.tusharedownloader import TushareDownloader
# from finrl.finrl_meta.preprocessor.processor_joinquant import JoinQuantEngineer
from finrl.finrl_meta.preprocessor.preprocessors import FeatureEngineer, data_split
# from finrl.finrl_meta.env_stock_trading.env_stocktrading import StockTradingEnv
from finrl.finrl_meta.env_stock_trading.env_test import StockTradingEnv

from finrl.agents.stablebaselines3.models import DRLAgent
from finrl.finrl_meta.data_processor import DataProcessor

from finrl.plot import backtest_stats, backtest_plot, get_daily_return, get_baseline
from pprint import pprint

import sys
sys.path.append("../FinRL-Library")

import itertools
import jqdatasdk
jqdatasdk.auth('15857500957','Qazwsxedcrfv0957')

/usr/local/lib/python3.7/dist-packages/pyfolio/pos.py:27: UserWarning: Module "zipline.assets" not found; multipliers will not be applied to position notionals.
  'Module "zipline.assets" not found; multipliers will not be applied'


auth success 


<a id='1.4'></a>
## 2.4. Create Folders

<a id='2'></a>
# Part 3. Download Data
Yahoo Finance is a website that provides stock data, financial news, financial reports, etc. All the data provided by Yahoo Finance is free.
* FinRL uses a class **YahooDownloader** to fetch data from Yahoo Finance API
* Call Limit: Using the Public API (without authentication), you are limited to 2,000 requests per hour per IP (or up to a total of 48,000 requests a day).




-----
class YahooDownloader:
    Provides methods for retrieving daily stock data from
    Yahoo Finance API

    Attributes
    ----------
        start_date : str
            start date of the data (modified from config.py)
        end_date : str
            end date of the data (modified from config.py)
        ticker_list : list
            a list of stock tickers (modified from config.py)

    Methods
    -------
    fetch_data()
        Fetches data from yahoo API


In [ ]:
# from config.py start_date is a string
config.TRAIN_START_DATE

'2014-01-01'

In [ ]:
# from config.py end_date is a string
config.TEST_END_DATE

'2021-10-01'

In [ ]:
df = YahooDownloader(start_date = '2019-01-01',
                     end_date = '2021-10-31',
                     ticker_list = config_tickers.DOW_30_TICKER).fetch_data()

[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%********

In [ ]:
print(config_tickers.DOW_30_TICKER)

['AXP', 'AMGN', 'AAPL', 'BA', 'CAT', 'CSCO', 'CVX', 'GS', 'HD', 'HON', 'IBM', 'INTC', 'JNJ', 'KO', 'JPM', 'MCD', 'MMM', 'MRK', 'MSFT', 'NKE', 'PG', 'TRV', 'UNH', 'CRM', 'VZ', 'V', 'WBA', 'WMT', 'DIS', 'DOW']


In [ ]:
df.shape

(21367, 8)

In [ ]:
df.sort_values(['date','tic'],ignore_index=True).head()

,date,open,high,low,close,volume,tic,day
0,2019-01-02,38.722500,39.712502,38.557499,38.277527,148158800,AAPL,2
1,2019-01-02,192.520004,193.199997,188.949997,174.294479,3009100,AMGN,2
2,2019-01-02,93.910004,96.269997,93.769997,91.087929,4175400,AXP,2
3,2019-01-02,316.190002,323.950012,313.709991,314.645142,3292200,BA,2
4,2019-01-02,124.029999,127.879997,123.000000,116.297020,4783200,CAT,2


In [ ]:
ndf = TushareDownloader(start_date = '2019-01-01',
                     end_date = '2021-10-31',
                     ticker_list = ['601398.SH', '600000.SH','601818.SH','600050.SH','300750.SZ']).fetch_data()
ndf

  0%|          | 0/5 [00:00<?, ?it/s]

本接口即将停止更新，请尽快使用Pro版接口：https://tushare.pro/document/2


 20%|██        | 1/5 [00:00<00:03,  1.01it/s]

本接口即将停止更新，请尽快使用Pro版接口：https://tushare.pro/document/2


 40%|████      | 2/5 [00:01<00:02,  1.23it/s]

...

本接口即将停止更新，请尽快使用Pro版接口：https://tushare.pro/document/2


100%|██████████| 5/5 [00:04<00:00,  1.23it/s]


Shape of DataFrame:  (2505, 9)
            date   open   high  close    low      volume  turnover     tic  \
0     2019-10-09  70.01  70.03  69.10  68.38   132218.56      1.10  300750   
1     2019-10-10  70.07  71.64  70.90  69.76   122851.54      1.02  300750   
2     2019-10-11  71.17  72.14  71.46  70.33    94925.62      0.79  300750   
3     2019-10-14  71.90  72.00  70.85  70.50   122035.88      1.01  300750   
4     2019-10-15  70.85  72.10  70.97  70.66    89350.77      0.74  300750   
...          ...    ...    ...    ...    ...         ...       ...     ...   
2500  2021-10-25   4.71   4.73   4.72   4.69  1033424.00      0.04  601398   
2501  2021-10-26   4.72   4.74   4.70   4.69  1187350.00      0.04  601398   
2502  2021-10-27   4.70   4.71   4.70   4.65  1257712.12      0.05  601398   
2503  2021-10-28   4.68   4.70   4.67   4.67   783008.44      0.03  601398   
2504  2021-10-29   4.67   4.69   4.67   4.66  1149808.00      0.04  601398   

      day  
0       2  
1       

/content/drive/MyDrive/HYF/Graduation_Fin/FinRL/finrl/finrl_meta/preprocessor/tusharedownloader.py:65: FutureWarning: In a future version of pandas all arguments of DataFrame.drop except for the argument 'labels' will be keyword-only
  data_df = data_df.drop(["price_change","p_change","ma5","ma10","ma20","v_ma5","v_ma10","v_ma20"], 1)


,date,open,high,close,low,volume,turnover,tic,day
0,2019-10-09,70.01,70.03,69.10,68.38,132218.56,1.10,300750,2
1,2019-10-09,11.82,12.13,11.99,11.80,422096.03,0.15,600000,2
2,2019-10-09,6.00,6.07,6.06,5.96,475071.66,0.22,600050,2
3,2019-10-09,5.53,5.60,5.58,5.53,1231203.38,0.05,601398,2
4,2019-10-09,4.07,4.30,4.23,4.06,2877277.25,0.72,601818,2
...,...,...,...,...,...,...,...,...,...
2500,2021-10-29,615.29,645.49,639.22,598.94,202687.02,1.00,300750,4
2501,2021-10-29,8.96,9.00,8.94,8.93,359907.31,0.12,600000,4
2502,2021-10-29,4.04,4.09,4.08,4.04,696518.00,0.23,600050,4
2503,2021-10-29,4.67,4.69,4.67,4.66,1149808.00,0.04,601398,4


In [ ]:
ticks=[('601398.XSHG', '600000.XSHG'),
 ('601398.XSHG', '600050.XSHG'),
 ('601398.XSHG', '601818.XSHG'),
 ('600000.XSHG', '600050.XSHG')]
mydf=jqdatasdk.get_price(ticks[0],'2021-01-01','2021-12-31').drop('money',axis=1)
mydf=mydf.rename(columns={'time':'date','code':'tic',})
mydf=mydf.sort_values(['date','tic'],ignore_index=True)
mydf['day']=pd.DataFrame(data=np.zeros(len(mydf),dtype=np.int8))

/usr/local/lib/python3.7/dist-packages/jqdatasdk/api.py:34: PanelObsoleteWarning: 当前环境 pandas 版本高于 0.25，get_price 与 get_fundamentals_continuously 接口的 panel 参数将固定为 False（0.25 及以上版本的 pandas 不再支持 panel，如使用该数据结构和相关函数请注意修改）
  warnings.warn(PandasChecker.VERSION_NOTICE_MESSAGE, PanelObsoleteWarning)


In [ ]:
mydf

,date,tic,open,close,high,low,volume,day
0,2021-01-04,600000.XSHG,9.18,9.22,9.26,9.09,66082052.0,0
1,2021-01-04,601398.XSHG,4.71,4.75,4.75,4.69,373335881.0,0
2,2021-01-05,600000.XSHG,9.21,9.21,9.21,9.06,56577620.0,0
3,2021-01-05,601398.XSHG,4.73,4.71,4.73,4.68,391329348.0,0
4,2021-01-06,600000.XSHG,9.16,9.35,9.36,9.16,65004667.0,0
...,...,...,...,...,...,...,...,...
481,2021-12-29,601398.XSHG,4.66,4.66,4.67,4.64,97664653.0,0
482,2021-12-30,600000.XSHG,8.54,8.54,8.58,8.53,23272484.0,0
483,2021-12-30,601398.XSHG,4.65,4.64,4.66,4.63,141275058.0,0
484,2021-12-31,600000.XSHG,8.54,8.53,8.57,8.53,22929621.0,0


In [ ]:
t=mydf
d=np.array(sorted(set(t['date'].dt.date)))
t['day']=t['date'].apply(lambda x:np.where(d==x)[0][0])
mydf=t
mydf

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:3: FutureWarning: Comparison of Timestamp with datetime.date is deprecated in order to match the standard library behavior.  In a future version these will be considered non-comparable.Use 'ts == pd.Timestamp(date)' or 'ts.date() == date' instead.
  This is separate from the ipykernel package so we can avoid doing imports until


,date,tic,open,close,high,low,volume,day
0,2021-01-04,600000.XSHG,9.18,9.22,9.26,9.09,66082052.0,0
1,2021-01-04,601398.XSHG,4.71,4.75,4.75,4.69,373335881.0,0
2,2021-01-05,600000.XSHG,9.21,9.21,9.21,9.06,56577620.0,1
3,2021-01-05,601398.XSHG,4.73,4.71,4.73,4.68,391329348.0,1
4,2021-01-06,600000.XSHG,9.16,9.35,9.36,9.16,65004667.0,2
...,...,...,...,...,...,...,...,...
481,2021-12-29,601398.XSHG,4.66,4.66,4.67,4.64,97664653.0,240
482,2021-12-30,600000.XSHG,8.54,8.54,8.58,8.53,23272484.0,241
483,2021-12-30,601398.XSHG,4.65,4.64,4.66,4.63,141275058.0,241
484,2021-12-31,600000.XSHG,8.54,8.53,8.57,8.53,22929621.0,242


In [ ]:
mydf=mydf.sort_values(['date','tic'],ignore_index=True)
mydf

,date,tic,open,close,high,low,volume,day
0,2021-01-04,600000.XSHG,9.18,9.22,9.26,9.09,66082052.0,0
1,2021-01-04,601398.XSHG,4.71,4.75,4.75,4.69,373335881.0,0
2,2021-01-05,600000.XSHG,9.21,9.21,9.21,9.06,56577620.0,1
3,2021-01-05,601398.XSHG,4.73,4.71,4.73,4.68,391329348.0,1
4,2021-01-06,600000.XSHG,9.16,9.35,9.36,9.16,65004667.0,2
...,...,...,...,...,...,...,...,...
481,2021-12-29,601398.XSHG,4.66,4.66,4.67,4.64,97664653.0,240
482,2021-12-30,600000.XSHG,8.54,8.54,8.58,8.53,23272484.0,241
483,2021-12-30,601398.XSHG,4.65,4.64,4.66,4.63,141275058.0,241
484,2021-12-31,600000.XSHG,8.54,8.53,8.57,8.53,22929621.0,242


# Part 4: Preprocess Data
Data preprocessing is a crucial step for training a high quality machine learning model. We need to check for missing data and do feature engineering in order to convert the data into a model-ready state.
* Add technical indicators. In practical trading, various information needs to be taken into account, for example the historical stock prices, current holding shares, technical indicators, etc. In this article, we demonstrate two trend-following technical indicators: **MACD and RSI.**
* Add turbulence index. Risk-aversion reflects whether an investor will choose to preserve the capital. It also influences one's trading strategy when facing different market volatility level. To control the risk in a worst-case scenario, such as financial crisis of 2007–2008, FinRL employs the financial turbulence index that measures extreme asset price fluctuation.

In [ ]:
fe = FeatureEngineer(
                    use_technical_indicator=True,
                    tech_indicator_list = config.INDICATORS,
                    use_vix=True,
                    use_turbulence=False,
                    user_defined_feature = False)

# processed = fe.preprocess_data(df)
processed = fe.preprocess_data(ndf)

Successfully added technical indicators
[*********************100%***********************]  1 of 1 completed
Shape of DataFrame:  (519, 8)
Successfully added vix


In [ ]:
list_ticker = processed["tic"].unique().tolist()
list_date = list(pd.date_range(processed['date'].min(),processed['date'].max()).astype(str))
combination = list(itertools.product(list_date,list_ticker))

processed_full = pd.DataFrame(combination,columns=["date","tic"]).merge(processed,on=["date","tic"],how="left")
processed_full = processed_full[processed_full['date'].isin(processed['date'])]
processed_full = processed_full.sort_values(['date','tic'])

processed_full = processed_full.fillna(0)

In [ ]:
processed_full.sort_values(['date','tic'],ignore_index=True).head(10)

,date,tic,open,high,close,low,volume,turnover,day,macd,boll_ub,boll_lb,rsi_30,cci_30,dx_30,close_30_sma,close_60_sma,vix
0,2019-10-09,300750,70.01,70.03,69.10,68.38,132218.56,1.10,2.0,0.000000,72.545584,67.454416,100.0,66.666667,100.0,69.100,69.100,18.639999
1,2019-10-09,600000,11.82,12.13,11.99,11.80,422096.03,0.15,2.0,0.000000,72.545584,67.454416,100.0,66.666667,100.0,11.990,11.990,18.639999
2,2019-10-09,600050,6.00,6.07,6.06,5.96,475071.66,0.22,2.0,0.000000,72.545584,67.454416,100.0,66.666667,100.0,6.060,6.060,18.639999
3,2019-10-09,601398,5.53,5.60,5.58,5.53,1231203.38,0.05,2.0,0.000000,72.545584,67.454416,100.0,66.666667,100.0,5.580,5.580,18.639999
4,2019-10-09,601818,4.07,4.30,4.23,4.06,2877277.25,0.72,2.0,0.000000,72.545584,67.454416,100.0,66.666667,100.0,4.230,4.230,18.639999
5,2019-10-10,300750,70.07,71.64,70.90,69.76,122851.54,1.02,3.0,0.040385,72.545584,67.454416,100.0,66.666667,100.0,70.000,70.000,17.570000
6,2019-10-10,600000,12.02,12.04,11.96,11.91,272612.28,0.10,3.0,-0.000673,12.017426,11.932574,0.0,-66.666667,100.0,11.975,11.975,17.570000
7,2019-10-10,600050,6.05,6.11,6.09,6.03,749842.75,0.35,3.0,0.000673,6.117426,6.032574,100.0,66.666667,100.0,6.075,6.075,17.570000
8,2019-10-10,601398,5.58,5.58,5.57,5.55,760767.44,0.03,3.0,-0.000224,5.589142,5.560858,0.0,-66.666667,100.0,5.575,5.575,17.570000
9,2019-10-10,601818,4.23,4.30,4.26,4.19,1336632.50,0.34,3.0,0.000673,4.287426,4.202574,100.0,66.666667,100.0,4.245,4.245,17.570000


In [ ]:
processed_full.to_csv('./datasets/train/processed_full.csv')

<a id='4'></a>
# Part 5. Design Environment
Considering the stochastic and interactive nature of the automated stock trading tasks, a financial task is modeled as a **Markov Decision Process (MDP)** problem. The training process involves observing stock price change, taking an action and reward's calculation to have the agent adjusting its strategy accordingly. By interacting with the environment, the trading agent will derive a trading strategy with the maximized rewards as time proceeds.

Our trading environments, based on OpenAI Gym framework, simulate live stock markets with real market data according to the principle of time-driven simulation.

The action space describes the allowed actions that the agent interacts with the environment. Normally, action a includes three actions: {-1, 0, 1}, where -1, 0, 1 represent selling, holding, and buying one share. Also, an action can be carried upon multiple shares. We use an action space {-k,…,-1, 0, 1, …, k}, where k denotes the number of shares to buy and -k denotes the number of shares to sell. For example, "Buy 10 shares of AAPL" or "Sell 10 shares of AAPL" are 10 or -10, respectively. The continuous action space needs to be normalized to [-1, 1], since the policy is defined on a Gaussian distribution, which needs to be normalized and symmetric.

## Training data split: 2009-01-01 to 2020-07-01
## Trade data split: 2020-07-01 to 2021-10-31

In [2]:
root='/content/drive/MyDrive/HYF/Graduation_Fin/FinRL'
%cd /content/drive/MyDrive/HYF/Graduation_Fin/FinRL
from finrl import config
from finrl import config_tickers
import os
if not os.path.exists(root+"/" + config.DATA_SAVE_DIR):
    os.makedirs(root+"/" + config.DATA_SAVE_DIR)
if not os.path.exists(root+"/" + config.TRAINED_MODEL_DIR):
    os.makedirs(root+"/" + config.TRAINED_MODEL_DIR)
if not os.path.exists(root+"/" + config.TENSORBOARD_LOG_DIR):
    os.makedirs(root+"/" + config.TENSORBOARD_LOG_DIR)
if not os.path.exists(root+"/" + config.RESULTS_DIR):
    os.makedirs(root+"/" + config.RESULTS_DIR)
import pandas as pd
import numpy as np
import matplotlib
import matplotlib.pyplot as plt
# matplotlib.use('Agg')
import datetime

%matplotlib inline
from finrl.finrl_meta.preprocessor.yahoodownloader import YahooDownloader
from finrl.finrl_meta.preprocessor.tusharedownloader import TushareDownloader
# from finrl.finrl_meta.preprocessor.processor_joinquant import JoinQuantEngineer
from finrl.finrl_meta.preprocessor.preprocessors import FeatureEngineer, data_split
# from finrl.finrl_meta.env_stock_trading.env_stocktrading import StockTradingEnv
from finrl.finrl_meta.env_stock_trading.env_test import StockTradingEnv

from finrl.agents.stablebaselines3.models import DRLAgent
from finrl.finrl_meta.data_processor import DataProcessor

from finrl.plot import backtest_stats, backtest_plot, get_daily_return, get_baseline
from pprint import pprint

import sys
sys.path.append("../FinRL-Library")

import itertools
import jqdatasdk
jqdatasdk.auth('15857500957','Qazwsxedcrfv0957')

/content/drive/MyDrive/HYF/Graduation_Fin/FinRL


/usr/local/lib/python3.7/dist-packages/pyfolio/pos.py:27: UserWarning: Module "zipline.assets" not found; mutltipliers will not be applied to position notionals.
  'Module "zipline.assets" not found; mutltipliers will not be applied' +
/usr/local/lib/python3.7/dist-packages/distributed/config.py:20: YAMLLoadWarning: calling yaml.load() without Loader=... is deprecated, as the default Loader is unsafe. Please read https://msg.pyyaml.org/load for full details.
  defaults = yaml.load(f)


auth success 


In [3]:
processed_full=pd.read_csv('./datasets/train/processed_full.csv',index_col=0,header=0)
processed_full

,date,tic,open,high,close,low,volume,turnover,day,macd,boll_ub,boll_lb,rsi_30,cci_30,dx_30,close_30_sma,close_60_sma,vix
0,2019-10-09,300750,70.01,70.03,69.10,68.38,132218.56,1.10,2.0,0.000000,72.545584,67.454416,100.000000,66.666667,100.000000,69.100000,69.100000,18.639999
1,2019-10-09,600000,11.82,12.13,11.99,11.80,422096.03,0.15,2.0,0.000000,72.545584,67.454416,100.000000,66.666667,100.000000,11.990000,11.990000,18.639999
2,2019-10-09,600050,6.00,6.07,6.06,5.96,475071.66,0.22,2.0,0.000000,72.545584,67.454416,100.000000,66.666667,100.000000,6.060000,6.060000,18.639999
3,2019-10-09,601398,5.53,5.60,5.58,5.53,1231203.38,0.05,2.0,0.000000,72.545584,67.454416,100.000000,66.666667,100.000000,5.580000,5.580000,18.639999
4,2019-10-09,601818,4.07,4.30,4.23,4.06,2877277.25,0.72,2.0,0.000000,72.545584,67.454416,100.000000,66.666667,100.000000,4.230000,4.230000,18.639999
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
3750,2021-10-28,300750,625.00,635.00,607.20,602.93,172828.69,0.85,3.0,26.432402,643.950436,468.972564,60.948071,130.122253,58.639166,539.562667,527.858167,16.530001
3751,2021-10-28,600000,8.99,9.01,8.96,8.95,281651.81,0.10,3.0,-0.044621,9.132375,8.948625,40.482541,-95.238095,0.470051,9.098000,9.134000,16.530001
3752,2021-10-28,600050,4.03,4.05,4.05,4.01,728765.44,0.24,3.0,-0.032678,4.265314,4.027686,41.862797,-157.410768,30.651240,4.179333,4.189667,16.530001
3753,2021-10-28,601398,4.68,4.70,4.67,4.67,783008.44,0.03,3.0,0.002753,4.774407,4.631593,45.710664,-50.997783,5.920235,4.706667,4.667000,16.530001


In [4]:
train = data_split(processed_full, '2009-01-01','2020-07-01')
trade = data_split(processed_full, '2020-07-01','2021-10-31')
print(len(train))
print(len(trade))

855
1570


In [5]:
train.tail()

,date,tic,open,high,close,low,volume,turnover,day,macd,boll_ub,boll_lb,rsi_30,cci_30,dx_30,close_30_sma,close_60_sma,vix
170,2020-06-30,300750,173.00,179.47,174.36,173.00,162514.33,1.33,1.0,7.031240,175.497841,143.159159,64.944704,177.894292,58.444181,155.013667,143.378833,30.43
170,2020-06-30,600000,10.59,10.64,10.58,10.54,228867.64,0.08,1.0,0.032445,10.754334,10.385666,51.101737,46.877965,23.535342,10.498000,10.377000,30.43
170,2020-06-30,600050,4.85,4.86,4.84,4.82,626092.62,0.29,1.0,-0.063535,5.125656,4.796344,39.196572,-138.872404,34.742841,4.994000,5.126500,30.43
170,2020-06-30,601398,5.00,5.03,4.98,4.96,2135599.00,0.08,1.0,-0.003584,5.343117,5.069883,40.554876,-198.985752,24.689803,5.173333,5.147167,30.43
170,2020-06-30,601818,3.58,3.60,3.58,3.54,1106169.50,0.28,1.0,-0.024317,3.948798,3.584202,41.962273,-173.655914,20.782707,3.750000,3.688667,30.43


In [6]:
trade.head()

,date,tic,open,high,close,low,volume,turnover,day,macd,boll_ub,boll_lb,rsi_30,cci_30,dx_30,close_30_sma,close_60_sma,vix
0,2020-07-01,300750,178.50,181.90,169.40,167.88,286574.78,2.34,2.0,6.854341,176.512623,143.989377,61.491314,142.407953,43.019601,155.933333,144.195667,28.620001
0,2020-07-01,600000,10.59,10.76,10.74,10.55,366907.09,0.13,2.0,0.045665,10.771046,10.377954,54.239666,96.316193,33.262973,10.512000,10.386833,28.620001
0,2020-07-01,600050,4.84,4.90,4.90,4.82,1069270.50,0.50,2.0,-0.059870,5.116403,4.791597,41.957818,-102.851942,23.997054,4.985333,5.121167,28.620001
0,2020-07-01,601398,4.98,4.99,4.99,4.92,1953424.25,0.07,2.0,-0.018126,5.364651,5.031349,41.065664,-204.998311,29.272906,5.169667,5.144500,28.620001
0,2020-07-01,601818,3.58,3.94,3.84,3.57,4483752.00,1.13,2.0,-0.011495,3.948798,3.584202,54.001259,26.633841,26.803140,3.756000,3.692500,28.620001


In [7]:
config.INDICATORS

['macd',
 'boll_ub',
 'boll_lb',
 'rsi_30',
 'cci_30',
 'dx_30',
 'close_30_sma',
 'close_60_sma']

In [8]:
stock_dimension = len(train.tic.unique())
state_space = 1 + 2*stock_dimension + len(config.INDICATORS)*stock_dimension
print(f"Stock Dimension: {stock_dimension}, State Space: {state_space}")


Stock Dimension: 5, State Space: 51


In [9]:
from finrl.finrl_meta.env_stock_trading.env_test import StockTradingEnv
env_kwargs = { 
        "hmax": 100, 
        "initial_amount": 1000000,
        "num_stock_shares": [0] * stock_dimension, 
        "buy_cost_pct": [0.000675] * stock_dimension, 
        "sell_cost_pct": [0.001] * stock_dimension, 
        "state_space": state_space, 
        "stock_dim": stock_dimension, 
        "tech_indicator_list": config.INDICATORS, 
        "action_space": stock_dimension, 
        "reward_scaling": 1e-4 }
e_train_gym = StockTradingEnv(df = train, **env_kwargs)

## Environment for Training



In [10]:
env_train, _ = e_train_gym.get_sb_env()
print(type(env_train))

<class 'stable_baselines3.common.vec_env.dummy_vec_env.DummyVecEnv'>


<a id='5'></a>
# Part 6: Implement DRL Algorithms
* The implementation of the DRL algorithms are based on **OpenAI Baselines** and **Stable Baselines**. Stable Baselines is a fork of OpenAI Baselines, with a major structural refactoring, and code cleanups.
* FinRL library includes fine-tuned standard DRL algorithms, such as DQN, DDPG,
Multi-Agent DDPG, PPO, SAC, A2C and TD3. We also allow users to
design their own DRL algorithms by adapting these DRL algorithms.

### Model Training: 5 models, A2C DDPG, PPO, TD3, SAC


### Model 3: PPO

In [11]:
agent = DRLAgent(env = env_train)

PPO_PARAMS = {
    "n_steps": 2048,
    "ent_coef": 0.01,
    "learning_rate": 0.00025,
    "batch_size": 128,
}
model_ppo = agent.get_model("ppo",model_kwargs = PPO_PARAMS)

{'n_steps': 2048, 'ent_coef': 0.01, 'learning_rate': 0.00025, 'batch_size': 128}
Using cuda device


In [12]:
trained_ppo = agent.train_model(model=model_ppo, 
                    tb_log_name='ppo',
                    total_timesteps=5000)

day: 170, episode: 10
begin_total_asset: 1000000.00
end_total_asset: 1053086.29
total_reward: 53086.29
total_cost: 1088.66
total_trades: 795
Sharpe: 2.153
-----------------------------------
| time/              |            |
|    fps             | 470        |
|    iterations      | 1          |
|    time_elapsed    | 4          |
|    total_timesteps | 2048       |
| train/             |            |
|    reward          | 0.38252375 |
-----------------------------------
day: 170, episode: 20
begin_total_asset: 1000000.00
end_total_asset: 1070851.70
total_reward: 70851.70
total_cost: 1203.31
total_trades: 797
Sharpe: 1.737
-----------------------------------------
| time/                   |             |
|    fps                  | 439         |
|    iterations           | 2           |
|    time_elapsed         | 9           |
|    total_timesteps      | 4096        |
| train/                  |             |
|    approx_kl            | 0.005526106 |
|    clip_fraction        | 0.

## other algos

### Model 1: A2C


In [13]:
# agent = DRLAgent(env = env_train)
# model_a2c = agent.get_model("a2c")

In [14]:
# trained_a2c = agent.train_model(model=model_a2c, 
#                              tb_log_name='a2c',
#                              total_timesteps=50000)

### Model 2: DDPG

In [15]:
# agent = DRLAgent(env = env_train)
# model_ddpg = agent.get_model("ddpg")

In [16]:
# trained_ddpg = agent.train_model(model=model_ddpg, 
#                              tb_log_name='ddpg',
#                              total_timesteps=50000)

### Model 4: TD3

In [17]:
# agent = DRLAgent(env = env_train)
# TD3_PARAMS = {"batch_size": 100, 
#               "buffer_size": 1000000, 
#               "learning_rate": 0.001}

# model_td3 = agent.get_model("td3",model_kwargs = TD3_PARAMS)

In [18]:
# trained_td3 = agent.train_model(model=model_td3, 
#                              tb_log_name='td3',
#                              total_timesteps=30000)

### Model 5: SAC

In [19]:
# agent = DRLAgent(env = env_train)
# SAC_PARAMS = {
#     "batch_size": 128,
#     "buffer_size": 1000000,
#     "learning_rate": 0.0001,
#     "learning_starts": 100,
#     "ent_coef": "auto_0.1",
# }

# model_sac = agent.get_model("sac",model_kwargs = SAC_PARAMS)

In [20]:
# trained_sac = agent.train_model(model=model_sac, 
#                              tb_log_name='sac',
#                              total_timesteps=60000)

## Trading

## Trading
Assume that we have $1,000,000 initial capital at 2020-07-01. We use the DDPG model to trade Dow jones 30 stocks.

### Set turbulence threshold
Set the turbulence threshold to be greater than the maximum of insample turbulence data, if current turbulence index is greater than the threshold, then we assume that the current market is volatile

In [21]:
# data_risk_indicator = processed_full[(processed_full.date<'2020-07-01') & (processed_full.date>='2009-01-01')]
# insample_risk_indicator = data_risk_indicator.drop_duplicates(subset=['date'])
# insample_risk_indicator.vix.describe()
# insample_risk_indicator.vix.quantile(0.996)
# insample_risk_indicator.turbulence.describe()
# insample_risk_indicator.turbulence.quantile(0.996)

### Trade

DRL model needs to update periodically in order to take full advantage of the data, ideally we need to retrain our model yearly, quarterly, or monthly. We also need to tune the parameters along the way, in this notebook I only use the in-sample data from 2009-01 to 2020-07 to tune the parameters once, so there is some alpha decay here as the length of trade date extends. 

Numerous hyperparameters – e.g. the learning rate, the total number of samples to train on – influence the learning process and are usually determined by testing some variations.

In [22]:
#trade = data_split(processed_full, '2020-07-01','2021-10-31')
from finrl.finrl_meta.env_stock_trading.env_stocktrading import StockTradingEnv

e_trade_gym = StockTradingEnv(df = trade, turbulence_threshold = None,risk_indicator_col='vix', **env_kwargs)
# e_trade_gym = StockTradingEnv(df = trade, turbulence_threshold = 70,risk_indicator_col='vix', **env_kwargs)
# env_trade, obs_trade = e_trade_gym.get_sb_env()

In [23]:
trade.head()

,date,tic,open,high,close,low,volume,turnover,day,macd,boll_ub,boll_lb,rsi_30,cci_30,dx_30,close_30_sma,close_60_sma,vix
0,2020-07-01,300750,178.50,181.90,169.40,167.88,286574.78,2.34,2.0,6.854341,176.512623,143.989377,61.491314,142.407953,43.019601,155.933333,144.195667,28.620001
0,2020-07-01,600000,10.59,10.76,10.74,10.55,366907.09,0.13,2.0,0.045665,10.771046,10.377954,54.239666,96.316193,33.262973,10.512000,10.386833,28.620001
0,2020-07-01,600050,4.84,4.90,4.90,4.82,1069270.50,0.50,2.0,-0.059870,5.116403,4.791597,41.957818,-102.851942,23.997054,4.985333,5.121167,28.620001
0,2020-07-01,601398,4.98,4.99,4.99,4.92,1953424.25,0.07,2.0,-0.018126,5.364651,5.031349,41.065664,-204.998311,29.272906,5.169667,5.144500,28.620001
0,2020-07-01,601818,3.58,3.94,3.84,3.57,4483752.00,1.13,2.0,-0.011495,3.948798,3.584202,54.001259,26.633841,26.803140,3.756000,3.692500,28.620001


In [24]:
df_account_value, df_actions = DRLAgent.DRL_prediction(
    model=trained_ppo, 
    # model=trained_a2c, 
    environment = e_trade_gym)

hit end!


In [25]:
df_account_value.shape

(314, 2)

In [26]:
df_account_value.tail()

,date,account_value
309,2021-10-22,2.091810e+06
310,2021-10-25,2.124238e+06
311,2021-10-26,2.079944e+06
312,2021-10-27,2.138979e+06
313,2021-10-28,2.111774e+06


In [27]:
df_actions.head()

,300750,600000,600050,601398,601818
date,,,,,
2020-07-01,16,0,0,7,3
2020-07-02,16,0,0,7,3
2020-07-06,16,0,0,7,3
2020-07-07,16,0,0,7,3
2020-07-08,16,0,0,7,3


<a id='6'></a>
# Part 7: Backtest Our Strategy
Backtesting plays a key role in evaluating the performance of a trading strategy. Automated backtesting tool is preferred because it reduces the human error. We usually use the Quantopian pyfolio package to backtest our trading strategies. It is easy to use and consists of various individual plots that provide a comprehensive image of the performance of a trading strategy.

<a id='6.1'></a>
## 7.1 BackTestStats
pass in df_account_value, this information is stored in env class


In [28]:
print("==============Get Backtest Results===========")
now = datetime.datetime.now().strftime('%Y%m%d-%Hh%M')

perf_stats_all = backtest_stats(account_value=df_account_value)
perf_stats_all = pd.DataFrame(perf_stats_all)
perf_stats_all.to_csv("./"+config.RESULTS_DIR+"/perf_stats_all_"+now+'.csv')

==============Get Backtest Results===========
Annual return          0.821986
Cumulative returns     1.111774
Annual volatility      0.391586
Sharpe ratio           1.732717
Calmar ratio           3.592823
Stability              0.852228
Max drawdown          -0.228786
Omega ratio            1.376327
Sortino ratio          2.828022
Skew                        NaN
Kurtosis                    NaN
Tail ratio             1.540264
Daily value at risk   -0.046643
dtype: float64


In [29]:
!pip list

Package                       Version
----------------------------- ---------------------
absl-py                       1.0.0
aiodns                        3.0.0
aiohttp                       3.8.1
aiohttp-cors                  0.7.0
aioredis                      1.3.1
aiosignal                     1.2.0
alabaster                     0.7.12
albumentations                0.1.12
ale-py                        0.7.4
alpaca-trade-api              1.2.3
altair                        4.2.0
appdirs                       1.4.4
argon2-cffi                   21.3.0
argon2-cffi-bindings          21.2.0
arviz                         0.11.4
astor                         0.8.1
astropy                       4.3.1
astunparse                    1.6.3
async-timeout                 4.0.2
asynctest                     0.13.0
atari-py                      0.2.9
atomicwrites                  1.4.0
attrs                         21.4.0
audioread                     2.1.9
autograd                      1.3
AutoR

In [30]:
#baseline stats
print("==============Get Baseline Stats===========")
baseline_df = get_baseline(
        ticker="^DJI", 
        start = df_account_value.loc[0,'date'],
        end = df_account_value.loc[len(df_account_value)-1,'date'])

stats = backtest_stats(baseline_df, value_col_name = 'close')


==============Get Baseline Stats===========
[*********************100%***********************]  1 of 1 completed
Shape of DataFrame:  (335, 8)
Annual return          0.273520
Cumulative returns     0.379084
Annual volatility      0.139248
Sharpe ratio           1.811893
Calmar ratio           3.062662
Stability              0.918651
Max drawdown          -0.089308
Omega ratio            1.351851
Sortino ratio          2.684720
Skew                        NaN
Kurtosis                    NaN
Tail ratio             1.051856
Daily value at risk   -0.016542
dtype: float64


In [31]:
df_account_value.loc[0,'date']

'2020-07-01'

In [32]:
df_account_value.loc[len(df_account_value)-1,'date']

'2021-10-28'

<a id='6.2'></a>
## 7.2 BackTestPlot

In [36]:
!pip install pyfolio==0.9.0

  Using cached pyfolio-0.9.0.tar.gz (91 kB)
  Created wheel for pyfolio: filename=pyfolio-0.9.0-py3-none-any.whl size=88933 sha256=5d6b7b4eb271766672af2128478c76721132e4a9f1e051fdd71decee4ebddd52
  Stored in directory: /root/.cache/pip/wheels/ec/45/ab/2869e6f72d2a77bf6e1c18186cd815ca53b855cc77e496ecb5
Successfully built pyfolio
  Attempting uninstall: pyfolio
    Found existing installation: pyfolio 0.9.1
    Uninstalling pyfolio-0.9.1:
      Successfully uninstalled pyfolio-0.9.1


In [37]:
print("penalty of buying")
print("==============Compare to DJIA===========")
%matplotlib inline
# S&P 500: ^GSPC
# Dow Jones Index: ^DJI
# NASDAQ 100: ^NDX
backtest_plot(df_account_value, 
             baseline_ticker = '^DJI', 
             baseline_start = df_account_value.loc[0,'date'],
             baseline_end = df_account_value.loc[len(df_account_value)-1,'date'])

penalty of buying
==============Compare to DJIA===========
[*********************100%***********************]  1 of 1 completed
Shape of DataFrame:  (335, 8)


Start date,2020-07-01
End date,2021-10-28
Total months,14
,Backtest
Annual return,82.2%
Cumulative returns,111.2%
Annual volatility,39.2%
Sharpe ratio,1.73
Calmar ratio,3.59
Stability,0.85
Max drawdown,-22.9%


AttributeError: ignored

In [ ]:
print("==============Compare to DJIA===========")
%matplotlib inline
# S&P 500: ^GSPC
# Dow Jones Index: ^DJI
# NASDAQ 100: ^NDX
backtest_plot(df_account_value, 
             baseline_ticker = '^DJI', 
             baseline_start = df_account_value.loc[0,'date'],
             baseline_end = df_account_value.loc[len(df_account_value)-1,'date'])

In [ ]:
df_actions

In [ ]:
np.argsort([-0.1783969 , 0.7176521])

In [ ]:
agent.env.get_attr('actions_memory')

In [ ]:
import numpy as np
actions=np.array([-70,  31,  -7, -91,   0])
argsort_actions = np.argsort(actions)#array([3, 0, 2, 4, 1])
argsort_actions[: np.where(actions < 0)[0].shape[0]]

In [ ]:
argsort_actions

In [ ]:
np.where(actions < 0)